# Importings

- consider imputing some of the tests
- take the LGBM or Catboost Performance o
- try different values for clipping

In [1]:
import pandas as pd                                    
import numpy as np        
import matplotlib.pyplot as plt
import seaborn as sns                              
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupKFold
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

class CFG:
  data_folder ="../khwarizm/data/"

def reading_data(path: str) -> pd.DataFrame:
  train = pd.read_csv(path + 'Train.csv')
  test = pd.read_csv(path + 'Test.csv')
  return train,test

train,test = reading_data(CFG.data_folder)

groups = train['ID']
test_id = test['ID_Zindi']
train_date = train['Date']
test_date = test['Date']
test_id = test['ID_Zindi']
pd.options.display.max_columns = 200

seed = 7 
seed2 = 77
tiney_fraction = 1e-05


In [2]:
train = train.dropna(subset=['GT_NO2'])
train.isnull().sum()

ID_Zindi                  0
Date                      0
ID                        0
LAT                       0
LON                       0
Precipitation             0
LST                   37594
AAI                   12118
CloudFraction         12118
NO2_strat             12118
NO2_total             12118
NO2_trop              33429
TropopausePressure    12118
GT_NO2                    0
dtype: int64

# Feature Engineering

In [3]:
groups = train['ID']
num_feats = train.select_dtypes(include=['float'])
kmeans = KMeans(n_clusters=2)

In [4]:
lat_min, lat_max = 44.92469405, 45.88973369
lon_min, lon_max = 8.736496578, 12.59068235

num_clusters_lat = 3
num_clusters_lon = 4
lat_step = (lat_max - lat_min) / num_clusters_lat
lon_step = (lon_max - lon_min) / num_clusters_lon
def assign_clusters(row, lat_step, lon_step, lat_min, lon_min):
    lat_cluster = int((row['LAT'] - lat_min) / lat_step)
    lon_cluster = int((row['LON'] - lon_min) / lon_step)
    return lat_cluster, lon_cluster
for dataset in (train, test):
    dataset[['lat_cluster', 'lon_cluster']] = dataset.apply(
        assign_clusters, axis=1, result_type='expand',
        lat_step=lat_step, lon_step=lon_step, lat_min=lat_min, lon_min=lon_min)

In [5]:
from khwarizm.time_series import time_features
"""
this is to get the time related features like year, month, week, day
"""
for df in (train,test):
    time_features(df)

train.columns

/workspaces/khwarizm/khwarizm/time_series.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataset['date'] = pd.to_datetime(dataset[col])
/workspaces/khwarizm/khwarizm/time_series.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataset['date'] = pd.to_datetime(dataset[col])


Index(['ID_Zindi', 'Date', 'ID', 'LAT', 'LON', 'Precipitation', 'LST', 'AAI',
       'CloudFraction', 'NO2_strat', 'NO2_total', 'NO2_trop',
       'TropopausePressure', 'GT_NO2', 'lat_cluster', 'lon_cluster', 'date',
       'Year', 'month', 'day', 'Weekday', 'Year_week', 'month_day'],
      dtype='object')

In [6]:
"""class Features:
    def __init__(self, dataset:pd.DataFrame,trainset: pd.DataFrame, testset: pd.DataFrame, date_feature: str):
        self.dataset = dataset
        self.trainset = trainset
        self.testset = testset
        self.date_feature = date_feature

    def rolling_feature( self,feature: str, window: int):
        for dataset in (self.trainset, self.testset):
            dataset[f"{feature}_rolling_max_{window}"] = dataset[feature].rolling(window).max()
            return dataset"""

'class Features:\n    def __init__(self, dataset:pd.DataFrame,trainset: pd.DataFrame, testset: pd.DataFrame, date_feature: str):\n        self.dataset = dataset\n        self.trainset = trainset\n        self.testset = testset\n        self.date_feature = date_feature\n\n    def rolling_feature( self,feature: str, window: int):\n        for dataset in (self.trainset, self.testset):\n            dataset[f"{feature}_rolling_max_{window}"] = dataset[feature].rolling(window).max()\n            return dataset'

In [7]:
from khwarizm.time_series import Features

df = pd.concat([train, test])
features = Features(dataset=df, trainset=train, testset=test, date_feature='Date')
train, test = features.rolling_feature(feature='NO2_trop', window=60)

In [8]:
df =pd.concat([train,test])
from khwarizm.time_series import Features
Features = Features(dataset=df,trainset=train,testset=test,date_feature='Date')
Features.rolling_feature('NO2_trop', window=60)

(        ID_Zindi      Date     ID        LAT        LON  Precipitation  \
 0      ID_ENTGC7    1/1/19   PD01  45.601585  11.903551       0.000000   
 1      ID_8JCCXC    1/1/19   PD04  45.371005  11.840830       3.047342   
 2      ID_V3136Z    1/1/19   RO01  45.045825  12.060869       0.000000   
 3      ID_KRVZDJ    1/1/19   RO02  45.104075  11.553241       1.200467   
 4      ID_PR351A    1/1/19   RO03  45.038758  11.790152       1.274564   
 ...          ...       ...    ...        ...        ...            ...   
 86579  ID_NCWXIY  31-12-21  X9897  45.498227   9.556232       0.000000   
 86580  ID_UDQIEE  31-12-21  X9928  45.142541  10.043836       0.000000   
 86581  ID_ENDUPX  31-12-21  X9969  45.842207   9.351658       0.000000   
 86582  ID_3ZBA6C  31-12-21  X9993  45.113503   8.874065       0.000000   
 86583  ID_GWG0LD  31-12-21  X9999  45.526473   9.515980       0.000000   
 
           LST       AAI  CloudFraction  NO2_strat  NO2_total  NO2_trop  \
 0         NaN  0.23052

In [9]:
stop

NameError: name 'stop' is not defined

# Missing Values & Encoding

In [10]:
for df in(train,test):
    df.drop(columns=['Date','ID','Precipitation','CloudFraction','AAI','lat_cluster'], axis=1,inplace=True)

def encoder(train,test):
  le = LabelEncoder()
  for df in(train,test):
      for col in df.columns:
          if df[col].dtype == 'object':
              df[col] = le.fit_transform(df[col])
  return train,test

encoder(train,test)

(       ID_Zindi        LAT        LON     LST  NO2_strat  NO2_total  NO2_trop  \
 0         33346  45.601585  11.903551     NaN   0.000024   0.000117       NaN   
 1         19485  45.371005  11.840830     NaN   0.000024   0.000127       NaN   
 2         70785  45.045825  12.060869     NaN   0.000024   0.000086       NaN   
 3         47159  45.104075  11.553241     NaN   0.000024   0.000124       NaN   
 4         58617  45.038758  11.790152     NaN   0.000024   0.000116       NaN   
 ...         ...        ...        ...     ...        ...        ...       ...   
 86579     53090  45.498227   9.556232     NaN   0.000032   0.000643       NaN   
 86580     69188  45.142541  10.043836     NaN        NaN        NaN       NaN   
 86581     33315  45.842207   9.351658  284.98   0.000031   0.000153  0.000122   
 86582      9113  45.113503   8.874065     NaN   0.000031   0.000717       NaN   
 86583     38431  45.526473   9.515980     NaN   0.000032   0.000643       NaN   
 
        Tropop

In [23]:
from khwarizm.utils import DataValidationUtils
dvu = DataValidationUtils()
train = dvu.format_data_types(train)
test = dvu.format_data_types(test)

In [24]:
train.dtypes

ID_Zindi                            int64
LAT                               float64
LON                               float64
LST                               float64
NO2_strat                         float64
NO2_total                         float64
NO2_trop                          float64
TropopausePressure                float64
GT_NO2                            float64
lon_cluster                         int64
date                       datetime64[ns]
Year                                int32
month                               int32
day                                 int32
Weekday                             int32
Year_week                           int64
month_day                           int64
NO2_trop_rolling_max_60           float64
dtype: object

# CV and Modeling

In [25]:
RandomForestModel = LGBMRegressor()
from khwarizm.feature_selection import validate_model
validate_model(model=RandomForestModel,cv='GroupKFold', n_splits=5,dataset=train,target_col='GT_NO2', groups=groups)

train shape : (65206,), test shape:(16845,)


DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float32DType'>)

In [11]:
"""model = CatBoostRegressor(
    iterations=1000,        
    learning_rate=0.1,      
    depth=6,                
    loss_function='RMSE',   
    eval_metric='RMSE',     
    random_seed=seed,          
    verbose=100,
   
)
"""

#model = CatBoostRegressor(random_state=seed)
#model = XGBRegressor(random_state= 7)
model =  LGBMRegressor(random_state=7)
n_splits = 5
n = train['GT_NO2'].count()
num_bins = int(1 + np.log2(n))
cv = GroupKFold(n_splits=n_splits)

def validate(trainset, testset, target_col):

    model.fit(trainset.drop(columns=target_col), trainset[target_col])
    pred = model.predict(testset.drop(columns=target_col))
    valid_idx = testset[target_col].notna()
    valid_testset = testset[target_col][valid_idx]
    valid_pred = pred[valid_idx]
    print('std:', valid_testset.std())
    score = mean_squared_error(valid_testset, valid_pred, squared=False)
    print('score:', score)

    return score
stds = []
rmse = []

for train_idx, test_idx in cv.split(train.drop(columns='GT_NO2'), train['GT_NO2'], groups=groups):
    train_v, test_v = train.iloc[train_idx], train.iloc[test_idx]
    stds.append(test_v['GT_NO2'].std())
    rmse.append(validate(train_v, test_v, 'GT_NO2'))

print('RMSE:', np.array(rmse).mean())
print('RMSE std:', np.array(rmse).std())
print('Standard Deviations:', stds)
print('RMSEs Deviations:', rmse)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007191 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2971
[LightGBM] [Info] Number of data points in the train set: 65206, number of used features: 18
[LightGBM] [Info] Start training from score 23.997347
std: 17.06148359415839
score: 10.457133906490098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006702 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2973
[LightGBM] [Info] Number of data points in the train set: 65234, number of used features: 18
[LightGBM] [Info] Start training from score 24.169432


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std: 19.215238775849844
score: 10.664819793296964
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005468 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2970
[LightGBM] [Info] Number of data points in the train set: 65834, number of used features: 18
[LightGBM] [Info] Start training from score 24.290743


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std: 18.587381576876247
score: 10.61840582290597
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005712 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2970
[LightGBM] [Info] Number of data points in the train set: 65824, number of used features: 18
[LightGBM] [Info] Start training from score 24.928707


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std: 14.468189480112546
score: 9.036741834674645


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


KeyboardInterrupt: 

In [61]:
def generate_specifications(model_name, num_folds, model_params, validation_type='StratifiedGroupKFold', shuffle=True):
    specs = []
    specs.append(f"Model: {model_name}")
    specs.append(f"Validation Type: {validation_type}")
    specs.append(f"Number of Folds: {num_folds}")
    specs.append(f"Shuffle: {'Yes' if shuffle else 'No'}")
    
    # Adding model parameters
    specs.append("Model Parameters:")
    for param, value in model_params.items():
        specs.append(f"  {param}: {value}")
    
    # Adding any additional info (if needed)
    specs.append("\nAdditional Notes:")
    specs.append("  -  as the evaluation metric.")
    
    # Joining all specifications into a single string
    return "\n".join(specs)

# Example usage

specifications = generate_specifications(model_name=str(model)[1:10], num_folds=n_splits, model_params=model.get_params())
print(specifications)

Model: GBMRegres
Validation Type: StratifiedGroupKFold
Number of Folds: 5
Shuffle: Yes
Model Parameters:
  boosting_type: gbdt
  class_weight: None
  colsample_bytree: 1.0
  importance_type: split
  learning_rate: 0.1
  max_depth: -1
  min_child_samples: 20
  min_child_weight: 0.001
  min_split_gain: 0.0
  n_estimators: 100
  n_jobs: None
  num_leaves: 31
  objective: None
  random_state: 7
  reg_alpha: 0.0
  reg_lambda: 0.0
  subsample: 1.0
  subsample_for_bin: 200000
  subsample_freq: 0

Additional Notes:
  -  as the evaluation metric.


# inference

In [63]:

model.fit(train.drop(columns='GT_NO2'),train['GT_NO2'])
y_pred = model.predict(test)
#sub_df = pd.DataFrame({'id': test_id,'GT_NO2':y_pred})
#sub_df.to_csv('submission9277409514290884.csv', index=False)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001309 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3012
[LightGBM] [Info] Number of data points in the train set: 48596, number of used features: 18
[LightGBM] [Info] Start training from score 22.909476


ValueError: Number of features of the model must match the input. Model n_features_ is 18 and input n_features is 14

In [ ]:
plt.figure(figsize=(16, 7))
sns.lineplot(x=train_date, y=train.GT_NO2, color='blue', label='Train')
sns.lineplot(x=test_date, y=y_pred, color='red', label='Prediction')
plt.legend()
plt.show()

In [ ]:
importances = model.feature_importances_
names = model.feature_names_
fi = pd.DataFrame({'Feature': names,
                   'importances': importances})
fi = fi.sort_values(by='importances', ascending=False)

fi.plot(kind='bar', x='Feature', y='importances', legend=False, figsize=(10, 6))
plt.title('Feature Importances')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.show()

In [ ]:
train.columns